In [1]:
import pandas as pd
import numpy as np
from bertopic import BERTopic
import sqlite3
import sqllite_handler

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
conn = sqlite3.connect('tables/texts_69_76.db')
cur = conn.cursor()

res = cur.execute("SELECT TEXT FROM transcript")
fetched = res.fetchall()
free_text_list = list(map(lambda x: x[0], fetched))

topic_model = BERTopic.load("plots/topic_model_69_76")

In [3]:
pd.set_option('display.max_rows', None)
topic_model.get_document_info(free_text_list)

,Document,Topic,Name,Top_n_words,Probability,Representative_document
0,HENRY A. KISSINGER BRIEFING OF WHITE HOUSE STA...,-1,-1_soviet_president_said_summit,soviet - president - said - summit - soviets -...,0.000000,False
1,SUBJECT President Nixon’s Meeting with Congres...,-1,-1_soviet_president_said_summit,soviet - president - said - summit - soviets -...,0.000000,False
2,"[Omitted here is discussion on the economy, wa...",-1,-1_soviet_president_said_summit,soviet - president - said - summit - soviets -...,0.000000,False
3,PARTICIPANTS Soviet Ambassador Anatoli Dobryni...,18,18_pakistan_soviet_india_united,pakistan - soviet - india - united - said - pr...,0.773737,False
4,"On October 16, 1971, Assistant to the Presiden...",-1,-1_soviet_president_said_summit,soviet - president - said - summit - soviets -...,0.000000,False
5,Dear Mr. Secretary: I appreciated receiving yo...,-1,-1_soviet_president_said_summit,soviet - president - said - summit - soviets -...,0.000000,False
6,7916. Personal for the Secretary. 1. We seem t...,-1,-1_soviet_president_said_summit,soviet - president - said - summit - soviets -...,0.000000,False
7,PARTICIPANTS Ambassador Anatoliy Dobrynin Dr. ...,21,21_said_dobrynin_asked_meeting,said - dobrynin - asked - meeting - told - vie...,0.887758,False
8,"At a November 3, 1971, meeting of the interage...",-1,-1_soviet_president_said_summit,soviet - president - said - summit - soviets -...,0.000000,False
9,"PARTICIPANTS Dr. Henry A. Kissinger, Assistant...",15,15_said_dobrynin_soviet_asked,said - dobrynin - soviet - asked - moscow - su...,1.000000,False


In [4]:
topic_model.topic_sizes_

{0: 235,
 -1: 206,
 1: 101,
 2: 80,
 3: 57,
 4: 48,
 5: 43,
 6: 41,
 7: 34,
 8: 31,
 9: 21,
 11: 18,
 10: 18,
 12: 17,
 13: 16,
 14: 15,
 15: 15,
 16: 14,
 17: 13,
 18: 12,
 19: 12,
 20: 12,
 21: 12}

In [6]:
from transformers import pipeline

triplet_extractor = pipeline('text2text-generation', model='Babelscape/rebel-large', tokenizer='Babelscape/rebel-large')

In [13]:
txt ='Luis lives in Zurich.'

out_ = triplet_extractor(txt, return_tensors=True, return_text=False)[0]["generated_token_ids"]

In [14]:
extracted_text = triplet_extractor.tokenizer.batch_decode(out_['output_ids'][0])

extracted_triplets = extract_triplets(extracted_text[0])
print(extracted_triplets)

[{'head': 'Luis', 'type': 'work location', 'tail': 'Zurich'}]


In [4]:
# Function to parse the generated text and extract the triplets
def extract_triplets(text):
    triplets = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
    return triplets


In [44]:
import spacy
nlp = spacy.load('en_core_web_sm')

def remove_persons(txt):
    document = nlp(txt)

    edited_txt = ""
    for ent in document:
        
        if ent.ent_type_=='PERSON':
            if ent.whitespace_:
                edited_txt += 'Person'+ ' '
            else:
                edited_txt += 'Person'
        else:
            if ent.whitespace_:
                edited_txt += ent.text+ ' '
            else:
                edited_txt += ent.text
    
    return edited_txt

In [1]:
import pandas as pd
doc_df = pd.read_csv('tables/doc_69_76v30.csv')

In [2]:
txt_list = doc_df['text'].values

In [49]:
processed_txt_list = list(map(lambda x: remove_persons(x), txt_list))

In [56]:
processed_txt_list[10]

'1438. Subj: U.S. Expression of Concern to Senior Greek Military. State 43153. 1. I appreciate the substance and thrust of the Dept’s telegram 43153. The situation here continues to be discouraging. Interference by the hardline Person junta has weakened the Armed Forces and incapacitated civil government. Popular resentment is continuing to build up against repressive political and capricious economic policies. The inherent instability of this power structure portends further change, possibly accompanied by civil unrest. There is widespread belief that the US is somehow responsible for this unhappy state of affairs. 2. Quite apart from the obvious danger of open unrest, however, I am increasingly concerned about the evolving chauvinistic attitude of the Ioannides junta as indicated in intelligence reports. If this group succeeds in creating a puppet military high command in addition to a puppet civil government or takes over direct ruling power itself, I fear that the policies it will 

In [5]:
from keybert import KeyBERT

# Extract keywords
kw_model = KeyBERT()
keywords = kw_model.extract_keywords(txt_list[0])



/Users/gokberk/miniconda3/envs/ml4hc_project2/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


: 

: 

In [5]:
import pickle

with open("plots/free_text_list", "rb") as fp:
        free_text_list = pickle.load(fp)

In [11]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [13]:
sentences = ["Relations between Greece and Turkey began in the 1830s following Greece's formation after its declaration of independence from the Ottoman Empire. Modern relations began when Turkey declared its formation in 1923 following the defeat of the Ottoman Empire in World War I. \
Greece and Turkey have a rivalry with a history of events that have been used to justify their nationalism.[1][2] These events include the population exchange between Greece and Turkey, the Istanbul pogrom and Cypriot intercommunal violence. Greek-Turkish feuding was not a significant factor in international relations from 1930 to 1955, and during the Cold War, domestic and bipolar politics limited competitive behaviour against each other.[3][4] By the mid-1990s and later decades, the restraint on their rivalry was removed, and both nations had become each other's biggest security risk.[5][6]\
Control of the eastern Mediterranean and Aegean seas remain the basis of the countries' rivalry. Following the end of World War II, the UNCLOS treaty, the decolonisation of Cyprus, and the addition of the Dodecanese to Greece's territory have caused turbulence in the relationship. Several issues frequently affect their current relations, including territorial disputes over the sea and air, minority rights, and Turkey's relationship with the European Union (EU) and its member states—especially Cyprus.[7][8] Control of energy pipelines is also an increasing focus in their relations."]

In [14]:
for sentence in sentences:
    sid = SentimentIntensityAnalyzer()
    print(sentence)
    ss = sid.polarity_scores(sentence)
    for k in sorted(ss):
        print('{0}: {1}, '.format(k, ss[k]), end='')
    print()

Relations between Greece and Turkey began in the 1830s following Greece's formation after its declaration of independence from the Ottoman Empire. Modern relations began when Turkey declared its formation in 1923 following the defeat of the Ottoman Empire in World War I. Greece and Turkey have a rivalry with a history of events that have been used to justify their nationalism.[1][2] These events include the population exchange between Greece and Turkey, the Istanbul pogrom and Cypriot intercommunal violence. Greek-Turkish feuding was not a significant factor in international relations from 1930 to 1955, and during the Cold War, domestic and bipolar politics limited competitive behaviour against each other.[3][4] By the mid-1990s and later decades, the restraint on their rivalry was removed, and both nations had become each other's biggest security risk.[5][6]Control of the eastern Mediterranean and Aegean seas remain the basis of the countries' rivalry. Following the end of World War I